In [24]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [2]:
! pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.4 MB/s eta 0:00:00


In [25]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [26]:
response = llm.invoke("Qual a previsão do tempo em Recife-RN?")
print(response.content)

Desculpe, mas não consigo fornecer informações em tempo real, como a previsão do tempo. Recomendo que você consulte um site de meteorologia ou um aplicativo de clima para obter as informações mais atualizadas sobre o tempo em Recife-RN.


In [27]:
a = 18273840567
b = 12830192835

response = llm.invoke("Quanto é 18273840567*12830192835? Responda apenas a o resultado.")
print(response.content)
print()
print(f'Resposta esperada:', a*b)

234084204189203999595.

Resposta esperada: 234456898310655737445


In [28]:
import random
from langchain.tools import tool

@tool
def add(a: int, b: int) -> int:
    """Soma dois inteiros.

    Args:
        a: Primeiro inteiro
        b: Segundo inteiro
    """
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiplica dois inteiros.

    Args:
        a: Primeiro inteiro
        b: Segundo inteiro
    """
    return a * b

@tool
def get_weather(city: str) -> str:
    """Retorna um clima aleatório para uma cidade.

    Args:
        city: Nome da cidade
    """
    weather = random.choice(["ensolarado", "nublado", "chuvoso", "tempestuoso", "neve"])
    return weather

tools = [add, multiply, get_weather]
llm_with_tools = llm.bind_tools(tools)

In [29]:
get_weather.invoke({"city": "Recife-RN"})

'ensolarado'

In [30]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage("Qual a previsão do tempo em Recife-RN? E também quanto é 123*321?")]

ai_msg = llm_with_tools.invoke(messages)

print(f"ContentString: {ai_msg.content}")
print(f"ToolCalls: {ai_msg.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'get_weather', 'args': {'city': 'Recife-RN'}, 'id': 'call_GrNoupMCo5ay1r4VW1mpvZr1', 'type': 'tool_call'}, {'name': 'multiply', 'args': {'a': 123, 'b': 321}, 'id': 'call_FGeQ07vtfSCzTnqyx5JG0bVO', 'type': 'tool_call'}]


In [32]:
! pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.2/138.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 4.1 MB/s eta 0:00:00


In [33]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

In [34]:
response = agent_executor.invoke({
    "messages": [
        HumanMessage("Qual a previsão do tempo em Recife-RN? E também quanto é 123*321?")
    ]
})

print(response["messages"])

[HumanMessage(content='Qual a previsão do tempo em Recife-RN? E também quanto é 123*321?', additional_kwargs={}, response_metadata={}, id='915b523f-ffa3-4a91-b041-3069e7baeca8'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mOdQcN5RRuieR3tDjmAY4zEr', 'function': {'arguments': '{"city": "Recife-RN"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_kCX7yYDI3wiryLVQJvZtDSgK', 'function': {'arguments': '{"a": 123, "b": 321}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 145, 'total_tokens': 196, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bd83329f63', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-65533610-c420-463d-864d-e3a09c8a

In [35]:
print(response["messages"][-1].content)

A previsão do tempo em Recife-RN é de neve, o que é bastante incomum para a região. 

Além disso, o resultado de 123 * 321 é 39.483.


## Exercícios

### Exercício 1
Adicione uma função nas ferramentas do agente para criar arquivos de texto. Em seguida, teste o agente.

In [36]:
@tool
# Função para criar um arquivo de texto
def create_text_file(filename, content):
    """Cria um arquivo texto contendo o conteúdo desejado.

    Args:
        filename: Nome do arquivo a ser criado
        content: Conteúdo do arquivo
    """
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(content)
    return f"Arquivo '{filename}' criado com sucesso."

tools.append(create_text_file)
agent_executor_ex1 = create_react_agent(llm, tools)

# Testando o agente
response = agent_executor_ex1.invoke({
    "messages": [
        HumanMessage("Crie o arquivo meu_texto.txt com o conteúdo 'Este é o conteúdo do arquivo criado pela nova função do agente para o exercício 1.'")
    ]
})

print(response["messages"][-1].content)

O arquivo **meu_texto.txt** foi criado com sucesso, contendo o seguinte texto: "Este é o conteúdo do arquivo criado pela nova função do agente para o exercício 1."


### Exercício 2
Crie um agente que conte a quantidade de ocorrências de uma determinada letra em uma palavra. Em seguida, teste o agente.

In [39]:
@tool
# Agente para contar a quantidade de ocorrências de uma letra em uma palavra
def count_letter_in_word(word, letter):
    """Conta o número de letras em uma palavra.

    Args:
        word: Palavra
        letter: Letra
    """
    return word.count(letter)

tools.append(count_letter_in_word)
agent_executor_ex2 = create_react_agent(llm, tools)

# Testando o agente
response = agent_executor_ex2.invoke({
    "messages": [
        HumanMessage("Quntas letras S existem na palavra 'inconstitucionalissimamente'?")
    ]
})

print(response["messages"][-1].content)

Existem 3 letras "S" na palavra "inconstitucionalissimamente".
